<h1> Long-term Performance of IPO </h1>

In [ ]:
file = pd.read_csv('/Users/Documents/IPO.csv')
file['diff'] = (pd.to_numeric(file['Last Price']) - pd.to_numeric(file['IPO Price'])) / pd.to_numeric(file['IPO Price']) * 100

# Check on the maximum and minimum values
file['diff'].describe()

# Define parameters of histogram
ipo_hist, edges = np.histogram(df['diff'], 
                               bins = int(260/5), 
                               range = [-50, 210])

# Put the information in dataframe
ipo = pd.DataFrame({'diff': ipo_hist, 
                       'left': edges[:-1], 
                       'right': edges[1:]})

# Define color for positive and negative interval
color = ["blue" if a>=0 else "red" for a in ipo['left']]

# Column Data Source
ipo['ipo_diff'] = ['%d' % diff for diff in ipo['diff']]
ipo['ipo_interval'] = ['%d to %d percent' % (left, right) for left, right in zip(ipo['left'], ipo['right'])]
ipo['color'] = color

# Convert dataframe to column data source
ipo_src = ColumnDataSource(ipo)

# Add basic styling
def style(p):
    p.title.align = 'center'
    p.title.text_font_size = '18pt'
    p.xaxis.axis_label_text_font_size = '12pt'
    p.xaxis.major_label_text_font_size = '12pt'
    p.yaxis.axis_label_text_font_size = '12pt'
    p.yaxis.major_label_text_font_size = '12pt'
    
    return p

# Set up the figure
p = figure(plot_width = 600, plot_height = 600, title = 'Long-Term Performance of IPO', x_axis_label = 'Percentage Gain/Loss', y_axis_label = 'Number of IPO')

# Add a quad glyph with source
p.quad(bottom=0, top='diff', left='left', right='right', source=ipo_src,
       fill_color= 'color', line_color='black')

# Add style to the plot
styled_p = style(p)

# Add a hover tool referring to the formatted columns
hover = HoverTool(tooltips = [('% Change', '@ipo_interval'),
                              ('No. of IPO', '@ipo_diff')])

# Add the hover tool to the graph
styled_p.add_tools(hover)

# Save plot
output_file("longterm.html")
save(styled_p)

# Calculate the number of IPO producing positive, negative and zero return
def calculate(data):
    positive_sum = 0
    negative_sum = 0
    zero_sum = 0 
    for n in data:
        if n > 0:
            positive_sum += 1
        if n == 0:
            zero_sum += 1
        elif n < 0:
            negative_sum += 1

    return [positive_sum, zero_sum, negative_sum]

calculate(file['diff'])